In [17]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np

In [18]:
# Load the model
model = keras.models.load_model('./fight_detection_model.h5')

In [26]:
# Create a video capture object
cap = cv2.VideoCapture("./No_Fight/fno_fight1.mp4")

In [27]:
# Set the width and height of the video
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

False

In [28]:
# Create a background subtractor object
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

In [29]:
# Create a kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

In [30]:
# Create a list to store the bounding boxes of the detected fights
fight_boxes = []

In [31]:

# Start the video capture loop
while True:

    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame is None
    if frame is None:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(gray)

    # Perform morphological operations to remove noise
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    # Find the contours in the foreground mask
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours
    for contour in contours:

        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Check if the bounding box is large enough
        if w > 50 and h > 50:

            # Add the bounding box to the list of fight boxes
            fight_boxes.append([x, y, w, h])

    # Predict the class of each bounding box
    fight_predictions = model.predict(np.array([cv2.resize(frame[y:y + h, x:x + w], (32, 32)) for x, y, w, h in fight_boxes]))

    # print(fight_predictions)
    # Iterate over the predictions
    for i, prediction in enumerate(fight_predictions):

        # Get the class label
        class_label = np.argmax(prediction)

        # Check if the class label is fight
        if class_label == 1:

            # Draw a red rectangle around the bounding box
            cv2.rectangle(frame, (fight_boxes[i][0], fight_boxes[i][1]), (fight_boxes[i][0] + fight_boxes[i][2], fight_boxes[i][1] + fight_boxes[i][3]), (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Check if the user wants to quit
    if cv2.waitKey(1) == ord('q'):
        break


In [25]:
# Clean up
cap.release()
cv2.destroyAllWindows()